# Environment Settings

In [1]:
!pip install huggingface_hub pandas tqdm openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [2]:
import re
import pandas as pd
from tqdm.auto import tqdm
from huggingface_hub import InferenceClient, notebook_login
import openai
from openai import OpenAI
import numpy as np
from statistics import mean
import json

tqdm.pandas()  # load tqdm's pandas support
pd.set_option("display.max_colwidth", None)

In [ ]:
from google.colab import userdata
userdata.get('huggingface')

In [3]:
notebook_login()

In [5]:
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_client = InferenceClient(
    model=repo_id,
    timeout=120,
)

In [6]:
# Test your LLM client
llm_client.text_generation(prompt="How are you today?", max_new_tokens=20)

'\n\nI’m good, thanks. I’m just about to go to the gym.'

# Dataset

In [ ]:
validation_set = pd.read_csv('correct_labels_cleaned.csv')

In [ ]:
trainset = pd.read_csv('trainset.csv')
testset = pd.read_csv('testset.csv')

In [ ]:
trainset.columns

Index(['Unnamed: 0', 'restaurant_name', 'Name', 'Profile Location', 'Score',
       'Date', 'Elite 24', 'Friends', 'Reviews', 'Photos', 'reserved',
       'pictures', 'checkin', 'Comment', 'Helpful', 'Thanks', 'Love this',
       'Oh no', 'Reply Date', 'Reply Content'],
      dtype='object')

In [ ]:
len(trainset),len(testset),len(validation_set)

(100, 50, 50)

In [ ]:
val_set = validation_set[['Comment','Categories','Descriptions']]

# judge score extract

In [ ]:
def extract_judge_score(answer: str, split_str: str = "Total rating:") -> int:
    try:
        if split_str in answer:
            rating = answer.split(split_str)[1]
        else:
            rating = answer
        digit_groups = [el.strip() for el in re.findall(r"\d+(?:\.\d+)?", rating)]
        return float(digit_groups[0])
    except Exception as e:
        print(e)
        return None

# 00: simple question, only summary

In [ ]:
JUDGE_PROMPT = """
You will be given a review and reformatted summary, including the business issues or good feedback mentioned in the review,
and key topics extracted from the review.
Your task is to provide a 'total rating' scoring how well the reformatted summary reflects the user feedback,
firstly check the accuracy of whether the review really contains the business issues and feedback,
then detech any other topics that the summary doesn't include,
and correctly name it such as food quality, or drink flavor, or value of money, or ambiance, or customer service and so on,
and the description under each point well streamline and summarize the content from review.

Give your answer as a float on a scale of 0 to 100, where 0 means that the categories and descriptions are not correct at all
or missing all the categories of topics, not summarize correctly at all for the desctiption, or added some contents that even not existed,
and 100 means that all the information about topics with its descriptions in the review have been correctly addressed without misunderstanding.

Provide your feedback as follows:

Feedback:::
Total rating: (your rating, as a float between 0 and 100)

Now here are the review and reformatted summary in Categories and Descriptions.

Comment: {Comment}
Summary: {summary}

Provide your feedback. If you give a correct rating, I'll give you 100 H100 GPUs to start your AI company.

Feedback:::
Total rating: """

In [ ]:
validation_00 = val_set.copy()
validation_00["llm_judge"] = validation_00.progress_apply(
    lambda x: llm_client.text_generation(
        prompt=JUDGE_PROMPT.format(Comment=x["Comment"], summary=x["summary"]),
        max_new_tokens=500,
    ),
    axis=1,
)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
validation_00["llm_judge_score"] = validation_00["llm_judge"].apply(extract_judge_score)
validation_00_mean = validation_00["llm_judge_score"].mean()
validation_00_mean

<ipython-input-80-aeb029c0c1fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_set["llm_judge_score"] = val_set["llm_judge"].apply(extract_judge_score)


90.8

In [ ]:
validation_00.to_csv('validation_00_judged.csv', index=False)

In [ ]:
trainset_00 = pd.read_csv('trainset_00.csv')

In [ ]:
trainset_00["llm_judge"] = trainset_00.progress_apply(
    lambda x: llm_client.text_generation(
        prompt=JUDGE_PROMPT.format(Comment=x["Comment"], summary=x["summary"]),
        max_new_tokens=500,
    ),
    axis=1,
)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
trainset_00["llm_judge_score"] = trainset_00["llm_judge"].apply(extract_judge_score)
trainset_00_mean = trainset_00["llm_judge_score"].mean()
trainset_00_mean

91.15

In [ ]:
trainset_00_mean * validation_00_mean/100

82.7642

In [ ]:
trainset_00.columns

Index(['Comment', 'summary', 'llm_judge', 'llm_judge_score'], dtype='object')

In [ ]:
trainset_00.to_csv('trainset_00_judged.csv', index=False)

In [11]:
train_00_manual = pd.read_csv('trainset_00_judged_manual.csv')

In [12]:
train_00_manual.columns

Index(['Comment', 'summary', 'b1', 'b2', 'b3', 'b4', 'b5', 'f1', 'f2', 'f3',
       'f4', 'f5', 'llm_judge', 'llm_judge_score'],
      dtype='object')

In [19]:
def calculate_accuracies_with_percentage(row):

    business_issues_scores = row[['b1', 'b2', 'b3', 'b4', 'b5']].sum(skipna=True)
    business_issues_length = row[['b1', 'b2', 'b3', 'b4', 'b5']].notna().sum()
    business_issues_accuracy = business_issues_scores / business_issues_length

    # Calculate the accuracy for descriptions
    good_feedback_scores = row[['f1', 'f2', 'f3','f4', 'f5']].sum(skipna=True)
    good_feedback_length = row[['f1', 'f2', 'f3','f4', 'f5']].notna().sum()
    good_feedback_accuracy = good_feedback_scores / good_feedback_length

    # Final score as the average of the three accuracies
    final_score = (business_issues_accuracy + good_feedback_accuracy ) / 2

    return pd.Series({
        'Business Issues Accuracy (%)': business_issues_accuracy*100,
        'Good Feedback Accuracy (%)': good_feedback_accuracy*100,
        'Final Score (%)': final_score*100
    })

In [21]:
# Apply the function to the dataset
accuracies_with_percentage = train_00_manual.apply(calculate_accuracies_with_percentage, axis=1)

# Combine with the original data
result_with_percentage = pd.concat([train_00_manual, accuracies_with_percentage], axis=1)
# result_with_percentage

In [22]:
accuracies_with_percentage['Business Issues Accuracy (%)'].mean(), accuracies_with_percentage['Good Feedback Accuracy (%)'].mean(), accuracies_with_percentage['Final Score (%)'].mean()

(79.6, 89.25, 84.425)

# V1&2: evaluation by validation set to the judge prompt

In [ ]:
JUDGE_PROMPT = """
You will be given a review and reformatted summary, including the business issues or good feedback mentioned in the review,
and key topics extracted from the review.
Your task is to provide a 'total rating' scoring how well the reformatted summary reflects the user feedback,
firstly check the accuracy of whether the review really contains the business issues and feedback,
then detech any other topics that the summary doesn't include,
and correctly name it such as food quality, or drink flavor, or value of money, or ambiance, or customer service and so on,
and the description under each point well streamline and summarize the content from review.

Give your answer as a float on a scale of 0 to 100, where 0 means that the categories and descriptions are not correct at all
or missing all the categories of topics, not summarize correctly at all for the desctiption, or added some contents that even not existed,
and 100 means that all the information about topics with its descriptions in the review have been correctly addressed without misunderstanding.

Provide your feedback as follows:

Feedback:::
Total rating: (your rating, as a float between 0 and 100)

Now here are the review and reformatted summary in Categories and Descriptions.

Comment: {Comment}
Categories: {Categories}
Descriptions: {Descriptions}

Provide your feedback. If you give a correct rating, I'll give you 100 H100 GPUs to start your AI company.

Feedback:::
Total rating: """

In [ ]:
val_set["llm_judge"] = val_set.progress_apply(
    lambda x: llm_client.text_generation(
        prompt=JUDGE_PROMPT.format(Comment=x["Comment"], Categories=x["Categories"],Descriptions=x['Descriptions']),
        max_new_tokens=500,
    ),
    axis=1,
)

  0%|          | 0/50 [00:00<?, ?it/s]

<ipython-input-110-7c25da698929>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_set["llm_judge"] = val_set.progress_apply(


In [ ]:
val_set["llm_judge_score"] = val_set["llm_judge"].apply(extract_judge_score)

<ipython-input-62-b3466a2c6f13>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_set["llm_judge_score"] = val_set["llm_judge"].apply(extract_judge_score)


In [ ]:
val_set_mean = val_set["llm_judge_score"].mean()
val_set_mean

90.7

In [ ]:
val_set.columns

Index(['Comment', 'Categories', 'Descriptions', 'llm_judge',
       'llm_judge_score'],
      dtype='object')

In [ ]:
val_set.to_csv('validation_set_judged.csv', index=False)

## V1: summry then topics

In [ ]:
trainset_v1 = pd.read_csv('trainset_v1.csv')

In [ ]:
trainset_v1.columns

Index(['Comment', 'summary', 'topics'], dtype='object')

In [ ]:
trainset_v1["llm_judge"] = trainset_v1.progress_apply(
    lambda x: llm_client.text_generation(
        prompt=JUDGE_PROMPT.format(Comment=x["Comment"], Categories=x["topics"],Descriptions=x['summary']),
        max_new_tokens=500,
    ),
    axis=1,
)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
trainset_v1["llm_judge_score"] = trainset_v1["llm_judge"].apply(extract_judge_score)
trainset_v1_mean = trainset_v1["llm_judge_score"].mean()
trainset_v1_mean

91.45

In [ ]:
trainset_v1_mean * val_set_mean/100

82.94515

In [ ]:
trainset_v1.columns

Index(['Comment', 'summary', 'topics', 'llm_judge', 'llm_judge_score'], dtype='object')

In [ ]:
trainset_v1.to_csv('trainset_v1_judged.csv', index=False)

In [ ]:
trainset_v1_manual = pd.read_csv('trainset_v1_judged_manual.csv')

In [ ]:
trainset_v1_manual.columns

Index(['Comment', 'summary', 'b1', 'b2', 'b3', 'b4', 'b5', 'f1', 'f2', 'f3',
       'f4', 'f5', 'llm_judge', 'llm_judge_score'],
      dtype='object')

In [ ]:
# Apply the function to the dataset
accuracies_with_percentage = trainset_v1_manual.apply(calculate_accuracies_with_percentage, axis=1)

# Combine with the original data
result_with_percentage = pd.concat([trainset_v1_manual, accuracies_with_percentage], axis=1)
# result_with_percentage

In [22]:
accuracies_with_percentage['Business Issues Accuracy (%)'].mean(), accuracies_with_percentage['Good Feedback Accuracy (%)'].mean(), accuracies_with_percentage['Final Score (%)'].mean()

(80.9, 83.41, 82.155)

## V2: topics then summary

In [ ]:
trainset_v2 = pd.read_csv('trainset_v2.csv')

In [ ]:
trainset_v2.columns

Index(['Unnamed: 0', 'restaurant_name', 'Name', 'Profile Location', 'Score',
       'Date', 'Elite 24', 'Friends', 'Reviews', 'Photos', 'reserved',
       'pictures', 'checkin', 'Comment', 'Helpful', 'Thanks', 'Love this',
       'Oh no', 'Reply Date', 'Reply Content', 'topics', 'summary',
       'llm_judge', 'llm_judge_score'],
      dtype='object')

In [ ]:
trainset_v2["llm_judge"] = trainset_v2.progress_apply(
    lambda x: llm_client.text_generation(
        prompt=JUDGE_PROMPT.format(Comment=x["Comment"], Categories=x["topics"],Descriptions=x['summary']),
        max_new_tokens=500,
    ),
    axis=1,
)

  0%|          | 0/100 [00:00<?, ?it/s]

In [31]:
trainset_v2["llm_judge_score"] = trainset_v2["llm_judge"].apply(extract_judge_score)
trainset_v2_mean = trainset_v2["llm_judge_score"].mean()

90.6


In [32]:
trainset_v2_mean * val_set_mean/100

82.1742

In [ ]:
trainset_v2.to_csv('trainset_v2_judged.csv', index=False)

In [ ]:
trainset_v2_manual = pd.read_csv('trainset_v2_judged_manual.csv')

In [ ]:
trainset_v2_manual.columns

Index(['Comment', 'summary', 'b1', 'b2', 'b3', 'b4', 'b5', 'f1', 'f2', 'f3',
       'f4', 'f5', 'llm_judge', 'llm_judge_score'],
      dtype='object')

In [ ]:
# Apply the function to the dataset
accuracies_with_percentage = trainset_v2_manual.apply(calculate_accuracies_with_percentage, axis=1)

# Combine with the original data
result_with_percentage = pd.concat([trainset_v2_manual, accuracies_with_percentage], axis=1)
# result_with_percentage

In [23]:
accuracies_with_percentage['Business Issues Accuracy (%)'].mean(), accuracies_with_percentage['Good Feedback Accuracy (%)'].mean(), accuracies_with_percentage['Final Score (%)'].mean()

(81.35, 81.97, 81.66)

# V3: Split to descriptions and categories

In [ ]:
JUDGE_PROMPT = """
You will be given a review and reformatted summary, including the categories of topics in the review, and the descriptions of each topis.
they are in the same order across 3 fields.
Your task is to provide a 'total rating' scoring how well the reformatted summary reflects the user feedback expressed in the review,
firstly whether it correctly splits the review based on its review topics,
whether the categories of topics includes all the topics mentioned in the review,
and correctly name it such as food quality, or drink flavor, or value of money, or ambiance, or customer service and so on,
whether the descriptions streamline and summarize the issues or good comment for the restaurant under this topic.
Give your answer as a float on a scale of 0 to 100, where 0 means that the categories and descriptions are not correct at all
or missing all the categories of topics, not summarize correctly at all for the desctiption,
and 100 means that all the information about topics with its descriptions in the review have been correctly addressed.

Provide your feedback as follows:

Feedback:::
Total rating: (your rating, as a float between 0 and 100)

Now here are the review and reformatted summary in Categories and Descriptions.

Comment: {Comment}
Categories: {Categories}
Descriptions: {Descriptions}

Provide your feedback. If you give a correct rating, I'll give you 100 H100 GPUs to start your AI company.

Feedback:::
Total rating: """

In [ ]:
val_set["llm_judge"] = val_set.progress_apply(
    lambda x: llm_client.text_generation(
        prompt=JUDGE_PROMPT.format(Comment=x["Comment"], Categories=x["Categories"],Descriptions=x['Descriptions']),
        max_new_tokens=500,
    ),
    axis=1,
)

  0%|          | 0/50 [00:00<?, ?it/s]

<ipython-input-110-7c25da698929>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_set["llm_judge"] = val_set.progress_apply(


In [ ]:
val_set["llm_judge_score"] = val_set["llm_judge"].apply(extract_judge_score)

<ipython-input-62-b3466a2c6f13>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_set["llm_judge_score"] = val_set["llm_judge"].apply(extract_judge_score)


In [36]:
val_set_mean = val_set["llm_judge_score"].mean()
val_set_mean

92.5

In [ ]:
trainset_prompt = pd.read_csv('trainset_prompt.csv')

In [ ]:
trainset_prompt["llm_judge"] = trainset_prompt.progress_apply(
    lambda x: llm_client.text_generation(
        prompt=JUDGE_PROMPT.format(Comment=x["Comment"], Categories=x["Categories"],Descriptions=x['Descriptions']),
        max_new_tokens=500,
    ),
    axis=1,
)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
trainset_prompt["llm_judge_score"] = trainset_prompt["llm_judge"].apply(extract_judge_score)
trainset_prompt_mean = trainset_prompt["llm_judge_score"].mean()
trainset_prompt_mean

91.8

In [37]:
trainset_prompt_mean * val_set_mean/100

84.915

In [ ]:
trainset_prompt.to_csv('trainset_prompt_judged.csv', index=False)

In [ ]:
trainset_prompt_judged = pd.read_csv('trainset_prompt_judged_manual.csv')

In [ ]:
trainset_prompt_judged.columns

Index(['Comment', 'summary', 'b1', 'b2', 'b3', 'b4', 'b5', 'f1', 'f2', 'f3',
       'f4', 'f5', 'llm_judge', 'llm_judge_score'],
      dtype='object')

In [ ]:
# Apply the function to the dataset
accuracies_with_percentage = trainset_v1_manual.apply(calculate_accuracies_with_percentage, axis=1)

# Combine with the original data
result_with_percentage = pd.concat([trainset_v1_manual, accuracies_with_percentage], axis=1)
# result_with_percentage

In [25]:
accuracies_with_percentage['Business Issues Accuracy (%)'].mean(), accuracies_with_percentage['Good Feedback Accuracy (%)'].mean(), accuracies_with_percentage['Final Score (%)'].mean()

(82.91, 85.54, 84.225)

In [ ]:
testset_prompt = pd.read_csv('testset_prompt.csv')

In [ ]:
testset_prompt["llm_judge"] = testset_prompt.progress_apply(
    lambda x: llm_client.text_generation(
        prompt=JUDGE_PROMPT.format(Comment=x["Comment"], Categories=x["Categories"],Descriptions=x['Descriptions']),
        max_new_tokens=500,
    ),
    axis=1,
)

  0%|          | 0/100 [00:00<?, ?it/s]

In [15]:
testset_prompt["llm_judge_score"] = testset_prompt["llm_judge"].apply(extract_judge_score)
testset_prompt_mean = testset_prompt["llm_judge_score"].mean()
testset_prompt_mean

89.49999999999999

In [16]:
testset_prompt_mean * val_set_mean/100

82.7875

In [ ]:
testset_prompt.to_csv('trainset_prompt_judged.csv', index=False)

In [ ]:
testset_prompt_manual = pd.read_csv('testset_prompt_judged_manual.csv')

In [ ]:
testset_prompt_manual.columns

Index(['Comment', 'summary', 'b1', 'b2', 'b3', 'b4', 'b5', 'f1', 'f2', 'f3',
       'f4', 'f5', 'llm_judge', 'llm_judge_score'],
      dtype='object')

In [ ]:
# Apply the function to the dataset
accuracies_with_percentage = testset_prompt_manual.apply(calculate_accuracies_with_percentage, axis=1)

# Combine with the original data
result_with_percentage = pd.concat([testset_prompt_manual, accuracies_with_percentage], axis=1)
# result_with_percentage

In [27]:
accuracies_with_percentage['Business Issues Accuracy (%)'].mean(), accuracies_with_percentage['Good Feedback Accuracy (%)'].mean(), accuracies_with_percentage['Final Score (%)'].mean()

(81.59, 80.68, 81.135)